# Welcome to Exkaldi

In this section, we will train a n-grams language model and query it.

Althrough __Srilm__ is avaliable in exkaldi, we recommend __Kenlm__ toolkit.

In [ ]:
import exkaldi

import os
dataDir = "librispeech_dummy"

Firstly, prepare the lexicons. We have generated and saved a __LexiconBank__ object in file already (3_prepare_lexicons). So restorage it directly.

In [ ]:
lexFile = os.path.join(dataDir, "exp", "lexicons.lex")

lexicons = exkaldi.decode.graph.load_lex(lexFile)

lexicons

We will use training text corpus to train LM model. Even though we have prepared a transcription file in the data directory, we do not need the utterance-ID information at the head of each line, so we must take a bit of work to produce a new text.

We can lend a hand of the exkaldi __Transcription__ class.

In [ ]:
textFile = os.path.join(dataDir, "train", "text")

trans = exkaldi.load_transcription(textFile)

trans

In [ ]:
newTextFile = os.path.join(dataDir, "exp", "train_lm_text")

trans.save(fileName=newTextFile, discardUttID=True)

But actually, you don't need do this. If you use a __Transcription__ object to train the language model, the information of utterance ID will be discarded automatically.

Now we train a 2-grams model with __Kenlm__ backend. (__srilm__ backend is also avaliable.)

In [ ]:
arpaFile = os.path.join(dataDir, "exp", "2-gram.arpa")

exkaldi.lm.train_ngrams_kenlm(lexicons, order=2, text=trans, outFile=arpaFile, config={"-S":"20%"})

ARPA model can be transform to binary format in order to accelerate loading and reduce memory cost.  
Although __KenLm__ Python API supports reading ARPA format, but in exkaldi, we only expected KenLM Binary format.

In [ ]:
binaryLmFile = os.path.join(dataDir, "exp", "2-gram.binary")

exkaldi.lm.arpa_to_binary(arpaFile, binaryLmFile)

Use the binary LM file to initialize a Python KenLM n-grams object.

In [ ]:
model = exkaldi.lm.KenNGrams(binaryLmFile)

model

__KenNGrams__ is simple wrapper of KenLM python Model. Defaultly, N-grams up to 6 orders can be use. If you want to use training bigger N-Grams, change the configure when you install the exkaldi pypi package from our github.

You can query this model with a sentence.

In [ ]:
model.score_sentence("HELLO WORLD", bos=True, eos=True)

There is a example to compute the perplexity of test corpus in order to evaluate the language model.

In [ ]:
evalTrans = exkaldi.load_transcription( os.path.join(dataDir, "test", "text") )

score = model.perplexity(evalTrans)

score

In [ ]:
type(score)

___score___ is an exkaldi __Metric__ (a subclass of Python dict) object. We design a group of classes to hold Kaldi text format table and exkaldi own text format data.

__ListTable__: spk2utt, utt2spk, words, phones and so on.  
__Transcription__: transcription corpus, n-best decoding result and so on.  
__Metric__: AM score, LM score, LM perplexity, Sentence lengthes and so on.  

All these classes are subclasses of Python dict. They have some common and respective methods and attributes. 

In this case, for example, we can compute the average value of __Metric__.

In [ ]:
score.mean()

More precisely, the weighted average by the length os sentences.

In [ ]:
score.mean( weight= evalTrans.sentence_length() )

As the termination of this section, we generate the Grammar fst for futher steps.

In [ ]:
Gfile = os.path.join(dataDir, "exp", "G.fst")

exkaldi.decode.graph.make_G(lexicons, arpaFile, outFile=Gfile, order=2)